# 2. Autograd-자동미분

PyTorch의 모든 신경망의 중심에는 autograd 패키지가 있다. 먼저 이것을 가볍게 살펴본 뒤, 첫번째 신경망을 학습시켜보겠다.<br>(주의! 가볍지 않을 수 있음😑)<br/>


<code>autograd패키지</code>는 **Tensor의 모든 연산에 대해 자동 미분을 제공**한다. 이는 <code>실행-기반-정의(define-by-run)프레임워크</code>로, 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매단계마다 달라진다.<br/>
😃 더 간단한 용어로 몇 가지 예를 살펴보자.

## Tensor

📝**Note**<br/>

<code>**Tensor 클래스**</code><br/>

*  패키지의 중심에는 **torch.Tensor 클래스**가 있다. 만약 <code>.requires_grad</code>속성을 True로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기 시작한다.

    * <code>.requires_grad=True</code> <- 뒤에서 많이 보게 될 것이다.

    계산이 완료된 후 .backward()를 호출하여 모든 변화도(gradient)를 자동으로 계산할 수 있다. 이 Tensor의 변화도는 .grad 속성에 누적된다.

*  Tensor가 기록을 추적하는 것을 중단하려면, <code>.detach()</code>를 호출하여 연산 기록으로부터 분리(detach)하여 이후 연산들이 추적되는 것을 방지할 수 있다. 

*  기록을 추적하는 것과 메모리를 사용하는 것을 방지하기 위해, 코드 블럭을 <code>with torch.no_grad():</code>로 감쌀 수 있다. 특히 변화도(gradient)는 필요없지만, requires_grad=True가 설정되어 학습 가능한 매개변수를 갖는 모델을 평가(evaluate)할 때 유용하다.

<code>**Function 클래스**</code><br/>

*  Autograd 구현에서 매우 중요한 클래스가 하나 더 있다. 바로 **<code>Function 클래스<code>**이다.

*  Tensor와 Function은 서로 연결되어 있으며, 모든 연산 과정을 부호화(encode)하여 순환하지 않는 그래프(acyclic graph)를 생성한다. 각 tensor는 <code>.grad_fn속성</code>을 갖고 있는데, 이는 Tensor를 생성한 Function을 참조하고 있다.(단, 사용자가 만든 Tensor는 예외로, 이 때 grad_fn은 None)이다.<br/>
🤨 이 말은 도통 설명만 들어서 모르겠군...

*  도함수를 계산하기 위해서는 Tensor의 .backward()를 호출하면 된다. 만약 Tensor가 스칼라(scalar)인 경우(예. 하나의 요소 값만 갖는 등) 에는 인자를 정해줄 필요가 없다. 하지만 여러 개의 요소를 갖고 있을 때는 tensor의 모양 gradient의 인자로 지정할 필요가 있다.

# Tutorial
👨🏻‍💻 Tutorial Start!

In [2]:
import torch

torch를 생성하고 requires_grad=True를 설정하여 연산을 기록한다.

In [3]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


tensor에 연산을 수행한다. requires_grad=True를 설정했기 때문에 연산을 기록한다.

In [4]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y는 연산의 결과로 생성된 것이므로 grad_fn을 갖는다<br/>
grad_fn이 생성된다.

In [5]:
print(y.grad_fn)

y에 다른 연산을 수행한다.

In [6]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


<code>.requires_grad(...)</code> 등장

.requires_grad(...)는 기존 Tensor의 requires_grad 값을 바꿔치기 (in-place)하여 변경한다. 입력값이 지정되지 않으면 기본값은 False이다.<br/>
🤷🏻‍♂️ 뭔 말이냐... 20.02.19.Wed pm4:43<br/>


아래의 예제를 통해 살펴보자 pm6:52

In [10]:
a = torch.randn(2, 2)  # standard normal distribution (2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


🤔 Q1. requires_grad(...)함수가 False일 때와 True일 때, 무엇이 다른걸까?<br/>
🤔 Q2. grad_fn은 뭔가...?

✍️ A1. requires_grad=True를 설정하면 연산을 기록할 수 있다. True일 때, 연산을 기록할 수 있다면 False일 때는 연산을 기록할 수 없다는 것이겠지!<br/>
✍️ A2. requires_grad=True로 설정한 변수로 다른 연산을 할 경우 생성되는 것 같다.(확실하지는 않다.)<br/>


Reference<br/>
https://pytorch.org/tutorials/beginner/former_torchies/autograd_tutorial.html

**AUTOGRAD**<br/>
Autograd is now core torch package for automatic differenciation. It uses a tape based system for automatic differenciation.<br/>
In the forward phase, the autograd tape will remember all the operations it executed, and in the backward phase, it will replay the operations.<br/> 
Each variable has <code>a .grad_fn</code> attribute that references a function that has created a function (except for Tensors created by the user - these have None as .grad_fn).<br/>

분명 한국어를 더 잘하는데... 영어로 받아들이는게 더 잘 이해가 되다니...!<br/>
"각 변수에는 함수를 생성한 함수를 참조하는 .grad_fn 속성이 있다."<br/>
결국 이것도 forward와 backward에서 계산을 추적하고 기억하기 위함이 아닐까

각각의 변수들은 .grad_fn라는 속성값을 가지고 있다. 이 .grad_fn은 함수를 생성한 함수를 참조한다. 즉 각 변수에는 함수를 생성한 함수를 참조하는 .grad_fn 속성이 있다.

## Gradient(변화도)

이제 역전파(backprop)을 해보자. out은 하나의 스칼라 값만 갖고 있기 때문에, out.backward()는 out.backward(torch.tensor(1.))과 동일하다.

In [12]:
out.backward()

변화도 d(out)/dx를 출력한다.<br/>
해석:: x값이 아주 조금 증가했을 때의 변화량. 즉, 기울기

In [13]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


🤨 아... 지난번에 했을 때, 여기서 막혔던 기억이...<br/>
😃 밑러닝1을 한 번 독파하고 왔기에 패기있게 밀고나간다 -20.02.19.wed pm7:10-

일반적으로, torch.autograd는 벡터-야코비안 곱을 계산하는 엔진이다.(Jocobian Matrix란 수학적으로 i에 대한 y의 변화도를 나타낸다.)<br/>
 즉, 어떤 벡터 v=(v1v2⋯vm)T 에 대해 vT⋅J 을 연산합니다. 만약 v 가 스칼라 함수 l=g(y⃗ ) 의 기울기인 경우, v=(∂l∂y1⋯∂l∂ym)T 이며, **연쇄법칙(chain rule)에 따라 벡터-야코비안 곱은 x⃗  에 대한 l 의 기울기가 됩니다:**

🤨 말이 참 어렵죠?<br/>
😃 밑러닝 수치미분편을 참고하시면 좋을 것 같습니다

벡터-야코비안 곱의 이러한 특성은 스칼라가 아닌 출력을 갖는 모델에 외부 변화도를 제공(feed)하는 것을 매우 편리하게 해준다. 

이제 벡터-야코비안 곱의 예제를 살펴보자.

In [19]:
# torch x 선언
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([104.9465, 377.7879, 939.2187], grad_fn=<MulBackward0>)


이 경우 y는 더 이상 스칼라 값이 아니다. <code>torch.autograd</code>는 전체 야코비안을 직접 계산할수는 없지만, 벡터-야코비안 곱은 간단히 <code>backward</code>에 해당 벡터를 인자로 제공하여 얻을 수 있다.

In [20]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


또한 <code>with torch.no_grad():</code>로 코드 블럭을 감싸서 autograd가 <code>.requires_grad=True</code>인 Tensor들의 연산 기록을 추적하는 것을 멈출 수 있다.

In [24]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


## Reference:
* 한글: https://tutorials.pytorch.kr/beginner/blitz/autograd_tutorial.html
* 영문: https://pytorch.org/tutorials/beginner/former_torchies/autograd_tutorial.html
한글판과 영문판의 내용이 조금 다르다... 이래서 영어를 해야한다는...!